In [ ]:
# -*- Bayesian_NN_Recall.py -*-
"""
Created Oct 2020

@author: Timothy E H Allen / Alistair M Middleton
"""
#%%

# Import modules
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.compat.v2 as tf
import tensorflow_probability as tfp
import tqdm
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import random
import seaborn as sns
from sklearn.metrics import mean_absolute_error

tf.enable_v2_behavior()
tfd = tfp.distributions

# Define inputs
evaluation_data = "/content/drive/My Drive/new_set_fingerprint.csv"
model_path = "/content/drive/My Drive/Model/AR_quantitative_predictor"
prediction_output_location = "/content/drive/My Drive/Output/new_set_output.csv"

def negloglik(y, rv_y):
    return -rv_y.log_prob(y)


reconstructed_model = tf.keras.models.load_model(model_path, custom_objects={'negloglik': negloglik})

def read_dataset(input_data):
    df = pd.read_csv(input_data)
    X = df[df.columns[0:10000]].values
    Y = df[df.columns[10000]]
    return X, Y

# Load in data
test_x, test_y = read_dataset(evaluation_data)

# Make predictions
y_pred_list = []
for i in tqdm.tqdm(range(500)):
  y_pred = reconstructed_model.predict(test_x)
  y_pred_list.append(y_pred)

y_preds = np.concatenate(y_pred_list, axis=1)
y_mean = pd.DataFrame(np.mean(y_preds, axis=1))
y_sigma = pd.DataFrame(np.std(y_preds, axis=1))

# Plot experimental vs predicted values
plt.figure()
plt.scatter(test_y,y_mean, marker='o')
plt.show()

Y_res = pd.concat([y_mean,y_sigma], axis=1)
Y_out = pd.concat([test_y,Y_res], axis=1)

# Output predictions
pd.DataFrame(Y_out).to_csv(prediction_output_location)

#End the cycle
tf.keras.backend.clear_session()

# Endgame
print("END")